In [60]:
# import necessary libraries 
import pandas as pd 
import os 
import glob 
import numpy as np  # Assuming you have NumPy for creating the array

# use glob to get all the csv files  
# in the folder 
path = "/Users/axellemoortgat/Projects/SimulationP2PEnergy/Data" 

csv_file_path1 = path + "/AgoriaA_2022.csv"
csv_file_path2 = path + "/ec1.csv"




# Read the CSV files into Pandas DataFrame
columns_to_keep = ['Datum', 'Tijd', 'Actief', 'Inductief', 'Capacitief', 'CosPhi', 'Dag/Nacht']
df1 = pd.read_csv(csv_file_path1, sep=';', usecols=columns_to_keep)

columns_to_keep = ['Van Datum', 'Van Tijdstip', 'Tot Datum', 'Tot Tijdstip', 'Register', 'Volume', 'Validatiestatus']
df2 = pd.read_csv(csv_file_path2, sep=';', usecols=columns_to_keep)

df2['Volume'] = pd.to_numeric(df2['Volume'].str.replace(',', '.'))
df2['Van Datum'] = pd.to_datetime(df2['Van Datum'], format='%d-%m-%Y')


# Convert relevant columns to numeric types
df1['Actief'] = pd.to_numeric(df1['Actief'], errors='coerce')
df1['Inductief'] = pd.to_numeric(df1['Inductief'], errors='coerce')
df1['Capacitief'] = pd.to_numeric(df1['Capacitief'], errors='coerce')
df1['CosPhi'] = pd.to_numeric(df1['CosPhi'], errors='coerce')

# Handle missing or NaN values if needed (this will replace NaN with 0)
df1 = df1.fillna(0)

# Calculate the total power
df1['TotaalVermogen'] = (df1['Actief'] + (df1['Inductief']**2 + df1['Capacitief']**2)**0.5)

# Calculate the active power using the cosine of the phase angle
df1['Actief'] = df1['TotaalVermogen'] * df1['CosPhi']


bedrijf = df1[['Datum', 'Tijd','TotaalVermogen', 'Dag/Nacht']]
bedrijf = bedrijf[ (bedrijf['Dag/Nacht'] != 'W') ]



injectie_df2 = df2[df2['Register'].str.contains('Injectie') & (df2['Validatiestatus'] == 'Gevalideerd')]
df2_injectie_2023 = injectie_df2[injectie_df2['Van Datum'].dt.year == 2023]

# Display the result
print(df2_injectie_2023)
print(bedrijf)

       Van Datum Van Tijdstip   Tot Datum Tot Tijdstip        Register  \
16713 2023-01-01     00:00:00  01-01-2023     00:15:00  Injectie Nacht   
16715 2023-01-01     00:15:00  01-01-2023     00:30:00  Injectie Nacht   
16717 2023-01-01     00:30:00  01-01-2023     00:45:00  Injectie Nacht   
16719 2023-01-01     00:45:00  01-01-2023     01:00:00  Injectie Nacht   
16721 2023-01-01     01:00:00  01-01-2023     01:15:00  Injectie Nacht   
...          ...          ...         ...          ...             ...   
84479 2023-12-31     22:45:00  31-12-2023     23:00:00  Injectie Nacht   
84481 2023-12-31     23:00:00  31-12-2023     23:15:00  Injectie Nacht   
84483 2023-12-31     23:15:00  31-12-2023     23:30:00  Injectie Nacht   
84485 2023-12-31     23:30:00  31-12-2023     23:45:00  Injectie Nacht   
84487 2023-12-31     23:45:00  01-01-2024     00:00:00  Injectie Nacht   

       Volume Validatiestatus  
16713     0.0     Gevalideerd  
16715     0.0     Gevalideerd  
16717     0.0  

In [61]:
# Converteer de 'Datum' kolom naar het gewenste formaat
bedrijf['Datum'] = pd.to_datetime(bedrijf['Datum'], format='%d.%m.%Y')

# Update het jaar naar 2023
bedrijf['Datum'] = bedrijf['Datum'].apply(lambda x: x.replace(year=2023))

# Converteer de 'Datum' kolom naar het nieuwe formaat
bedrijf['Datum'] = bedrijf['Datum'].dt.strftime('%Y-%m-%d')

bedrijf = bedrijf.drop(bedrijf.index[-1])


# Toon het resultaat
print(bedrijf['Datum'])

184      2023-01-02
185      2023-01-02
186      2023-01-02
187      2023-01-02
188      2023-01-02
            ...    
34966    2023-12-31
34967    2023-12-31
34968    2023-12-31
34969    2023-12-31
34970    2023-12-31
Name: Datum, Length: 26815, dtype: object


In [62]:
# TOTAL INJECTIE/AFNAME 2023
total_sum_injectie_df2_2023 = df2_injectie_2023['Volume'].sum()
total_sum_afname_bedrijf_2023 = bedrijf['TotaalVermogen'].sum()

# Display the total sum
print(f'Total Sum Injectie: {total_sum_injectie_df2_2023}')
print(f'Total Sum Afname: {total_sum_afname_bedrijf_2023}')

Total Sum Injectie: 4265.8769999999995
Total Sum Afname: 10792423.884176433


In [63]:
bedrijf = bedrijf.rename(columns={'Datum': 'Van Datum', 'Tijd': 'Van Tijdstip'})

df2['Van Datum'] = pd.to_datetime(df2['Van Datum'])
bedrijf['Van Datum'] = pd.to_datetime(bedrijf['Van Datum'])



merged_df_2023 = pd.merge(bedrijf, df2_injectie_2023, on=['Van Datum', 'Van Tijdstip'])


In [64]:
merged_df_2023 = merged_df_2023.rename(columns={'TotaalVermogen': 'Afname', 'Volume': 'Injectie'})

# Create a new column for the subtraction result
merged_df_2023['P2PTransactie'] = merged_df_2023[['Injectie', 'Afname']].min(axis=1)
merged_df_2023['P2POverschot'] = merged_df_2023['Injectie'] - merged_df_2023['P2PTransactie']
print(merged_df_2023)


       Van Datum Van Tijdstip  Afname Dag/Nacht   Tot Datum Tot Tijdstip  \
0     2023-01-02     22:15:00    84.0         N  02-01-2023     22:30:00   
1     2023-01-02     22:30:00    98.0         N  02-01-2023     22:45:00   
2     2023-01-02     22:45:00    84.0         N  02-01-2023     23:00:00   
3     2023-01-02     23:00:00    84.0         N  02-01-2023     23:15:00   
4     2023-01-02     23:15:00    91.0         N  02-01-2023     23:30:00   
...          ...          ...     ...       ...         ...          ...   
25806 2023-12-31     05:45:00    56.0         N  31-12-2023     06:00:00   
25807 2023-12-31     06:00:00    56.0         N  31-12-2023     06:15:00   
25808 2023-12-31     06:15:00    63.0         N  31-12-2023     06:30:00   
25809 2023-12-31     06:30:00    49.0         N  31-12-2023     06:45:00   
25810 2023-12-31     06:45:00    63.0         N  31-12-2023     07:00:00   

             Register  Injectie Validatiestatus  P2PTransactie  P2POverschot  
0      I

In [65]:
# Display rows where P2POverschot is negative for merged_dag_df
rest_injection_2023 = merged_df_2023[merged_df_2023['P2POverschot'] > 0]
print("Rows with positive P2POverschot in Merged DataFrame for Dag:")
print(rest_injection_2023[['Van Datum', 'Van Tijdstip', 'Afname', 'Injectie', 'P2PTransactie', 'P2POverschot']])


Rows with positive P2POverschot in Merged DataFrame for Dag:
       Van Datum Van Tijdstip  Afname  Injectie  P2PTransactie  P2POverschot
9730  2023-05-26     08:45:00     0.0     0.002            0.0         0.002
9732  2023-05-26     09:15:00     0.0     0.004            0.0         0.004
9733  2023-05-26     09:30:00     0.0     0.121            0.0         0.121
9734  2023-05-26     09:45:00     0.0     0.936            0.0         0.936
9735  2023-05-26     10:00:00     0.0     1.260            0.0         1.260
...          ...          ...     ...       ...            ...           ...
25543 2023-12-28     12:00:00     0.0     0.001            0.0         0.001
25636 2023-12-29     11:15:00     0.0     0.003            0.0         0.003
25637 2023-12-29     11:30:00     0.0     0.003            0.0         0.003
25638 2023-12-29     11:45:00     0.0     0.001            0.0         0.001
25745 2023-12-30     14:30:00     0.0     0.002            0.0         0.002

[732 rows x 6 

In [66]:
total_not_transfered_2023 = rest_injection_2023['P2POverschot'].sum()
print(f'Total not transfered 2023: {total_not_transfered_2023}')

Total not transfered 2023: 391.264


In [67]:
total_transfered_2023 = total_sum_injectie_df2_2023-total_not_transfered_2023
print(f'Total transfered 2023: {total_transfered_2023}')

Total transfered 2023: 3874.6129999999994


In [68]:
print(f'Rendement 2023: {total_transfered_2023 /(total_transfered_2023+total_not_transfered_2023)}')

Rendement 2023: 0.9082805247314912
